In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
import pandas as pd
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def load_pickle(path):
    with open(path,'rb') as f:
        return pickle.load(f)

def write_pickle(path, d):
    with open(path,'wb+') as f:
        return pickle.dump(d, f, protocol = pickle.HIGHEST_PROTOCOL)

In [1]:
conv1 = load_pickle('./drive/MyDrive/hidden-networks/dataset/conv1_s20.pkl')
conv2 = load_pickle('./drive/MyDrive/hidden-networks/dataset/conv2_s20.pkl')
fc1 = load_pickle('./drive/MyDrive/hidden-networks/dataset/fc1_s20.pkl')
fc2 = load_pickle('./drive/MyDrive/hidden-networks/dataset/fc2_s20.pkl')

NameError: ignored

In [ ]:
conv1_masks, conv1_weights = conv1
conv1_masks = conv1_masks.squeeze()
conv1_weights = conv1_weights.squeeze()
conv2_masks, conv2_weights = conv2
conv2_masks = conv1_masks.squeeze()
conv2_weights = conv1_weights.squeeze()
fc1_masks, fc1_weights = fc1
fc2_masks, fc2_weights = fc2

In [ ]:
print(conv1 == conv2)

In [ ]:
def get_sign(weight):
  if weight > 0:
    return 1
  elif weight < 0:
    return -1
  else:
    return 0

In [ ]:
def process_weight(weight):
  return abs(weight).item(), get_sign(weight)

In [ ]:
def featurize_conv(weights, masks, sparsity):
  weights_padded =  F.pad(weights, (1,1,1,1), "constant", 0)
  data_conv = []
  for channel_num, channel in enumerate(weights_padded):
    for row in range(1, channel.shape[0] - 1):
      for col in range(1, channel.shape[1] - 1):
        mag_0, sign_0 = process_weight(channel[row][col])
        mag_1, sign_1 = process_weight(channel[row-1][col-1])
        mag_2, sign_2 = process_weight(channel[row-1][col])
        mag_3, sign_3 = process_weight(channel[row-1][col+1])
        mag_4, sign_4 = process_weight(channel[row][col-1])
        mag_5, sign_5 = process_weight(channel[row][col+1])
        mag_6, sign_6 = process_weight(channel[row+1][col-1])
        mag_7, sign_7 = process_weight(channel[row+1][col])
        mag_8, sign_8 = process_weight(channel[row+1][col+1])
        include = masks[channel_num][row-1][col-1].item()
        data_conv.append([mag_0, mag_1, mag_2, mag_3, mag_4, mag_5, mag_6, mag_7, mag_8, sign_0, sign_1, sign_2, sign_3, sign_4, sign_5, sign_6, sign_7, sign_8, sparsity, "conv", include])
  return data_conv

In [ ]:
conv1_data = featurize_conv(conv1_weights, conv1_masks, 0.2)
conv2_data = featurize_conv(conv2_weights, conv2_masks, 0.2)

In [ ]:
conv_data = conv1_data + conv2_data

In [ ]:
import csv
with open("./drive/MyDrive/hidden-networks/dataset/conv.csv", "a+", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(conv_data)

In [ ]:
df = pd.DataFrame(conv_data, columns = ['mag', 'mag_1','mag_2','mag_3','mag_4','mag_5','mag_6','mag_7','mag_8', 'sign', 'sign_1', 'sign_2', 'sign_3', 'sign_4', 'sign_5', 'sign_6', 'sign_7', 'sign_8', 'type', 'sparsity', 'include'])

In [ ]:
df

,mag,mag_1,mag_2,mag_3,mag_4,mag_5,mag_6,mag_7,mag_8,sign,...,sign_2,sign_3,sign_4,sign_5,sign_6,sign_7,sign_8,type,sparsity,include
0,0.418261,0.000000,0.000000,0.000000,0.000000,0.249749,0.000000,0.165046,0.128344,-1,...,0,0,0,1,0,1,-1,0.2,conv,0.0
1,0.249749,0.000000,0.000000,0.000000,0.418261,0.126525,0.165046,0.128344,0.502784,1,...,0,0,-1,1,1,-1,1,0.2,conv,0.0
2,0.126525,0.000000,0.000000,0.000000,0.249749,0.000000,0.128344,0.502784,0.000000,1,...,0,0,1,0,-1,1,0,0.2,conv,0.0
3,0.165046,0.000000,0.418261,0.249749,0.000000,0.128344,0.000000,0.422059,0.666942,1,...,-1,1,0,-1,0,-1,1,0.2,conv,0.0
4,0.128344,0.418261,0.249749,0.126525,0.165046,0.502784,0.422059,0.666942,0.432411,-1,...,1,1,1,1,-1,1,-1,0.2,conv,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,0.187238,0.406820,0.233080,0.543482,0.081545,0.211038,0.216270,0.347666,0.557253,1,...,1,1,1,-1,1,1,-1,0.2,conv,0.0
572,0.211038,0.233080,0.543482,0.000000,0.187238,0.000000,0.347666,0.557253,0.000000,-1,...,1,0,1,0,1,-1,0,0.2,conv,0.0
573,0.216270,0.000000,0.081545,0.187238,0.000000,0.347666,0.000000,0.000000,0.000000,1,...,1,1,0,1,0,0,0,0.2,conv,0.0
574,0.347666,0.081545,0.187238,0.211038,0.216270,0.557253,0.000000,0.000000,0.000000,1,...,1,-1,1,-1,0,0,0,0.2,conv,0.0


In [ ]:
df.to_csv('./drive/MyDrive/hidden-networks/dataset/conv.csv', mode='a') 

In [ ]:
import statsmodels.formula.api as smf

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
log_reg = smf.logit("include ~-1 + mag + mag_1 + mag_2 + mag_3+ mag_4 + mag_5+ mag_6+ mag_7+ mag_8 + sign + sign_1 + sign_2 + sign_3 + sign_4 + sign_5 + sign_6 + sign_7 + sign_8", data=df).fit()

Optimization terminated successfully.
         Current function value: 0.424750
         Iterations 7


In [ ]:
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                include   No. Observations:                  576
Model:                          Logit   Df Residuals:                      558
Method:                           MLE   Df Model:                           17
Date:                Wed, 06 Apr 2022   Pseudo R-squ.:                  0.1544
Time:                        05:34:42   Log-Likelihood:                -244.66
converged:                       True   LL-Null:                       -289.34
Covariance Type:            nonrobust   LLR p-value:                 7.969e-12
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
mag            2.3454      0.355      6.610      0.000       1.650       3.041
mag_1         -0.6177      0.488     -1.266      0.206      -1.574       0.339
mag_2         -2.6509      0.472     -5.619      0.000      -3.576      -1.726
mag_3         -2.1717      0.584     -3.720      0.000      -3.316      -1.027
mag_4         -0.2057      0.399     -0.516      0.606      -0.987       0.576
mag_5         -0.2705      0.438     -0.618      0.537      -1.129       0.588
mag_6         -2.1069      0.571     -3.688      0.000      -3.227      -0.987
mag_7         -2.0521      0.436     -4.708      0.000      -2.906      -1.198
mag_8         -2.4422      0.603     -4.050      0.000      -3.624      -1.260
sign           0.0575      0.118      0.488      0.625      -0.173       0.288
sign_1         0.3571      0.181      1.977      0.048       0.003       0.711
sign_2         0.2754      0.150      1.835      0.067      -0.019       0.570
sign_3         0.0342      0.196      0.174      0.862      -0.351       0.419
sign_4        -0.2578      0.145     -1.783      0.075      -0.541       0.026
sign_5        -0.0262      0.149     -0.175      0.861      -0.318       0.266
sign_6         0.1758      0.186      0.945      0.344      -0.189       0.540
sign_7         0.2306      0.149      1.543      0.123      -0.062       0.523
sign_8         0.3367      0.196      1.720      0.085      -0.047       0.720
==============================================================================
"""

In [ ]:
[1 ,2 ,3
 4, 0 ,5
 6, 7, 8]